In [13]:
import pandas as pd
import seaborn as sns
import json
import requests
import numpy as np

In [8]:
r = requests.get('http://localhost:5000/v1/stats/data/full')
df = pd.read_json(json.dumps(r.json()))

## Exploring the data

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103658 entries, 0 to 103657
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   age_6              19184 non-null   float64
 1   cma                103658 non-null  int64  
 2   cma_name           103658 non-null  object 
 3   cowmain            67062 non-null   float64
 4   cowmain_name       103658 non-null  object 
 5   educ               103658 non-null  int64  
 6   full_naics_21      103658 non-null  object 
 7   immig              103658 non-null  object 
 8   index              103658 non-null  int64  
 9   is_student         76893 non-null   float64
 10  lfsstat            103658 non-null  int64  
 11  naics_21           103658 non-null  object 
 12  prov               103658 non-null  int64  
 13  prov_name          103658 non-null  object 
 14  schooln            76893 non-null   float64
 15  sex                103658 non-null  object 
 16  st

### Dataset first look
The dataframe has a total of 18 different columns. Those columns only use the following types: `[object, int64 and float64]`. We can determine that the boolean values, for instance, `is_student` has been encoded to a float when importing the data in mysql.

### Null and Non-null values
A few columns with 103658 non-null values have null values at all since there is a total of 103658 entries.

It is interesting to note that the new columns we have created in the cleaning process have no non-null values, since the mapping from the source column includes setting a non-null value even when the source value in `NaN.

For instance, if the original value for `cowmain` is `NaN`, the corresponding `cowmain_name` would be `'Not applicable'`

### Other observations
We also note that the source data has some columns such as `age_6` which is missing source information since we would expect any person to have an age. This would probably be attributable to the inability to acquire this data during the data collection phase, when creating the dataset.

## Gender distribution

### Distribution of gender per province

In [118]:
data = df[['sex', 'prov_name']].groupby('prov_name').value_counts()
data

prov_name  sex
AB         F       4030
           M       3945
BC         F       6804
           M       6191
MB         F       4357
           M       4147
ND         F       2854
           M       2612
NL         F       2107
           M       1963
NS         F       2776
           M       2485
ON         F      16515
           M      15515
PE         F       1167
           M       1085
QC         F       9940
           M       9476
SK         F       2941
           M       2748
dtype: int64

### Distribution of gender in Quebec

In [116]:
data = df.query('prov_name == "QC"')['sex'].value_counts()
data

F    9940
M    9476
Name: sex, dtype: int64

## What is the labour force status, per province, for all the workers?

In [142]:
# 14,1,2017,, lfsstat    ,Labour force status,Situation d'activit�,,,,,,,,,
# ,,,,,,,,,,,1,"Employed, at work","Personnes occup�es, au travail",,
# ,,,,,,,,,,,2,"Employed, absent from work","Personnes occup�es, absentes du travail",,
# ,,,,,,,,,,,3,Unemployed,Ch�meurs,,
# ,,,,,,,,,,,4,Not in labour force,Inactifs,,

# We exclude lfsstat == 4 since this means the person is not in workforce, therefore not a worker
df.query('lfsstat != 4')[['lfsstat', 'prov_name']].groupby('prov_name').value_counts()

prov_name  lfsstat
AB         1           4603
           3            360
           2            351
BC         1           7066
           2            540
           3            386
MB         1           4782
           2            363
           3            236
ND         1           2581
           3            258
           2            249
NL         1           1863
           3            262
           2            193
NS         1           2513
           2            225
           3            214
ON         1          17136
           2           1392
           3           1051
PE         1           1135
           3            149
           2             96
QC         1          10369
           2            945
           3            540
SK         1           3228
           2            313
           3            174
dtype: int64